## Analyse bivariée : Coachs nationaux vs clubs étrangers

**Problématique :**  
> *Les coachs entraînent-ils plus souvent dans leur propre pays ?*

Nous allons croiser deux variables :
- la nationalité du coach (Italie ou non)
- le pays (ou “League”) du club coaché

L’objectif est d’identifier s’il existe un lien entre ces deux variables.  
Pour cela, nous utiliserons un **tableau croisé** et un **test du Chi²**.

Cette analyse permet d’observer la **mobilité internationale** des coachs dans leur carrière.


## Requête SQL – Extraction des coachs et des clubs

Nous avons utilisé cette requête SQL pour extraire les données nécessaires à notre analyse :

```sql
SELECT 
    c.name AS coachName,
    c.nationality,
    c.nationality2,
    t.Name AS teamName,
    t.League AS teamCountry
FROM mandates m
JOIN coaches c ON m.coach_qid = c.wikidata_id
JOIN teams t ON m.team_qid = t.Qid
WHERE t.League IS NOT NULL 
  AND (c.nationality IS NOT NULL OR c.nationality2 IS NOT NULL)


## Chargement du fichier CSV

Nous chargeons les données extraites de SQLiteStudio contenant :
- le nom du coach
- ses deux nationalités
- le club qu’il a coaché
- le pays (ou "league") de ce club

Nous allons ensuite créer deux variables catégorielles :
- `coach_is_italian` : True si une des nationalités est italienne
- `club_is_italian` : True si le pays du club est l’Italie


In [3]:
import pandas as pd

df = pd.read_csv("coach_vs_country.csv")

# Nettoyage des chaînes
df["nationality"] = df["nationality"].astype(str).str.strip()
df["nationality2"] = df["nationality2"].astype(str).str.strip()
df["teamCountry"] = df["teamCountry"].astype(str).str.strip()

# le coach est-il italien ?
df["coach_is_italian"] = df.apply(
    lambda row: "Italy" in [row["nationality"], row["nationality2"]],
    axis=1
)

# le club est-il italien ?
df["club_is_italian"] = df["teamCountry"] == "Italy"

df.head()


,coachName,nationality,nationality2,teamName,teamCountry,coach_is_italian,club_is_italian
0,Dino Zoff,Italy,nan,Juventus FC,Italy,True,True
1,Dino Zoff,Italy,nan,ACF Fiorentina,Italy,True,True
2,Dino Zoff,Italy,nan,SS Lazio,Italy,True,True
3,Dino Zoff,Italy,nan,SS Lazio,Italy,True,True
4,Dino Zoff,Italy,nan,SS Lazio,Italy,True,True


## Analyse croisée : coachs italiens vs clubs italiens

Nous allons construire un tableau croisé pour observer la relation entre :
- la nationalité du coach (italien ou non)
- le pays du club (italien ou non)

Nous appliquerons ensuite un **test du Chi²** pour tester l’indépendance statistique entre ces deux variables.


In [5]:
from scipy.stats import chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt

# Tableau croisé
contingency = pd.crosstab(df["coach_is_italian"], df["club_is_italian"])
print(" Tableau croisé :\n")
print(contingency)

# Chi²
chi2, p, dof, expected = chi2_contingency(contingency)

print("\n Résultats du test du Chi² :")
print(f"Statistique de test : {chi2:.2f}")
print(f"p-value : {p:.4f}")
print(f"Degrés de liberté : {dof}")


 Tableau croisé :

club_is_italian   False  True 
coach_is_italian              
False               209     88
True                112    506

 Résultats du test du Chi² :
Statistique de test : 238.15
p-value : 0.0000
Degrés de liberté : 1


## Conclusion de l’analyse

Le test du **Chi²** appliqué à la relation entre la **nationalité du coach** et le **pays du club coaché** donne les résultats suivants :

- **Statistique du test :** 238.15  
- **p-value :** < 0.0001  
- **Degrés de liberté :** 1

---

### Interprétation :

La p-value est **très inférieure à 0.05**, ce qui signifie que l'on peut **rejeter l’hypothèse d’indépendance** entre les deux variables.

Autrement dit, **la nationalité du coach influence significativement la probabilité qu’il entraîne dans son propre pays.**

Cela est également visible dans le tableau croisé :

- Parmi les **coachs italiens**, 506 mandats (soit environ **82%**) concernent des clubs italiens.
- Parmi les **coachs étrangers**, seulement 88 mandats (soit environ **30%**) concernent des clubs italiens.

---

### Limites de l’analyse :

Cette conclusion doit être interprétée avec prudence, car notre analyse présente plusieurs biais importants :

- **Population restreinte :** nous analysons uniquement des coachs ayant **entraîné en Serie A**, ce qui **exclut** :
  - les coachs italiens n’ayant **jamais** travaillé en Serie A
  - les étrangers n’ayant **jamais** entraîné en Italie
- **Mobilité sous-estimée :** un coach étranger dans notre échantillon a nécessairement **déjà exercé en Italie**, ce qui réduit artificiellement sa mobilité apparente.
- **Comptage par mandat et non par individu :** un coach peut apparaître plusieurs fois s’il a eu plusieurs mandats, ce qui surpondère les carrières longues.
- **Manque de complétude des données Wikidata :** certains mandats, notamment à l’étranger ou dans des clubs moins connus, peuvent être absents.

---

### Conclusion :

Même si l’analyse montre une **forte corrélation** entre la nationalité et le pays du club, elle ne reflète qu’une **vision partielle** du marché du coaching.  
Il serait intéressant de reproduire cette analyse avec des données plus complètes (UEFA, Transfermarkt) pour une vision globale de la mobilité.
